# Generics in Scala

![a](../misc/genericeverywhere.jpg)

Generics is a widely spread feature of many programming languages, and Scala is no exception. We can find it virtually in every Scala library we want to use: 

![lists](../misc/scala-collections-list.png)

![list-map](../misc/scala-api-list-map.png)

![Scala Collections](../misc/scala-api-list-prepended.png)

![spark-dataset](../misc/spark-api-dataset.png)

![dataset-flatmap](../misc/spark-api-dataset-flatmap.png)

![spark-groupbykey](../misc/spark-api-dataset-groupbykey.png)

![akka-flow](../misc/akka-api-flow.png)

![akka-flow-fold](../misc/akka-api-flow-foldasync.png)

![akka-flow-runwith](../misc/akka-api-flow-runwith.png)

# Generic features have a long history

The first programming language to implement them was ML (1975). Then, they come Haskell, C++, ... and, of course, Java: 

![java-generics](../misc/genericsjava.png)

![odersky](../misc/odersky.png)

![lambdaman](../misc/lambdaman.png)
Prof. Philip Wadler @LambdaWorld'16

# Generics, what and why?

Let's motivate the need for generics with a simple, related problem, which is not solved with generics, but it will serve as an inspiration anyway. We start from this definition of lists of integers: 

In [1]:
sealed abstract class IntList
case class End() extends IntList
case class Cons(head: Int, tail: IntList) extends IntList

defined class IntList
defined class End
defined class Cons

and a function to add the number five to the end of the list: 

In [2]:
def addFive(list: IntList): IntList = 
    list match {
        case End() => Cons(5, End())
        case Cons(head, tail) => Cons(head, addFive(tail))
    }

defined function addFive

There is something wrong with this function: it's too specific. What if we want to add a different number, say three? We may copy & paste the above definition and replace five with three: 

In [3]:
def addThree(list: IntList): IntList = 
    list match {
        case End() => Cons(3, End())
        case Cons(head, tail) => Cons(head, addThree(tail))
    }

defined function addThree

But, of course, we may rather abstract the function from the number to be inserted, and extends the signature with a new parameter. 

In [4]:
def addNumber(n: Int, list: IntList): IntList = 
    list match {
        case End() => Cons(n, End())
        case Cons(head, tail) => Cons(head, addNumber(n, tail))
    }

defined function addNumber

Indeed, that's the very purpose of functions: to obtain more modular programs.

In [5]:
def addFive(list: IntList): IntList = addNumber(5, list)
def addThree(list: IntList): IntList = addNumber(3, list)

defined function addFive
defined function addThree

But, did we finish here our modularization work? Not yet. Let's say that we now want a list of strings. What do we do, copy & paste again?

In [6]:
sealed abstract class StringList
case class End() extends StringList
case class Cons(head: String, tail: StringList) extends StringList

defined class StringList
defined class End
defined class Cons

and the same for the function `addString`?

In [7]:
def addString(n: String, list: StringList): StringList = 
    list match {
        case End() => Cons(n, End())
        case Cons(head, tail) => Cons(head, addString(n, tail))
    }

defined function addString

Couldn't we do something similar to what we did before? Yes, but now we have to abstract the definition of `addString` and `StringList` from the *type* `String`. This is how we get to our first generic declaration:

In [8]:
sealed abstract class List[A]
case class End[A]() extends List[A]
case class Cons[A](head: A, tail: List[A]) extends List[A]

defined class List
defined class End
defined class Cons

and we can do the same for the generic method definition:

In [9]:
def add[A](n: A, list: List[A]): List[A] = 
    list match {
        case End() => Cons(n, End())
        case Cons(head, tail) => Cons(head, add[A](n, tail))
    }

defined function add

Now, the `addNumber` and `addString` functions can be fully modularised as follows:

In [10]:
def addNumber(n: Int, l: List[Int]): List[Int] = 
    add[Int](n, l)
def addString(n: String, l: List[String]): List[String] = 
    add[String](n, l)

defined function addNumber
defined function addString

The important thing to remember is that we can treat value parameters and type parameters as just that, i.e. **parameters**. And when we invoke a generic method, we will normally pass values, but also types. There is a difference concerning types, however: types can be normally inferred by the Scala compiler, whereas values can not (except if they are declared implicitly, as we will see in the next session). Thus, we can invoke our generic function as follows:

In [11]:
add[Int](5, Cons(5, Cons(4, End())))

res10: List[Int] = Cons(5, Cons(4, Cons(5, End())))

but also in the following way: 

In [12]:
add(5, Cons(5, Cons(4, End())))

res11: List[Int] = Cons(5, Cons(4, Cons(5, End())))

# The power of  _parametric polymorphism_

The kind of generic programming illustrated above is called *parametric polymorphism*. It constrasts with other kinds of polymorphism such as *ad-hoc polymorphism* (which will be mentioned in the session about implicits), and *subtype polymorphism* (which will be exemplified later on in this session). 

In order to better undertand this kind of polymorphism we will consider a number of examples. For instance, let's consider the following monomorphic function. How many functions with this signature are there?

In [13]:
def foo(i: Int): Int = ???

defined function foo

a lot, actually: ${(2^{32})}^{2^{32}}$

In contrast, how many functions are there with the following polymorphic signature?

In [14]:
def foo[A](a: A): A = ???

defined function foo

Well, we may pretend to give different implementations doing horrible things, as for instance: 

In [15]:
def foo[A](a: A): A = 
    1.asInstanceOf[A]

defined function foo

This function compiles, but it won't always work at runtime: 

In [16]:
foo[Int](1)

res15: Int = 1

In [17]:
foo[String]("")

: 

Here it is another implementation, this time taking advantage that `Nothing` (the "type" of exceptions) is a subtype of every other type:

In [ ]:
def foo[A](a: A): A = throw new RuntimeException("error")

We may even try (and achieve) to dispatch a different behaviour depending on the type:

In [ ]:
def foo[A](a: A): A = 
    if (a.isInstanceOf[String]) ???
    else if (a.isInstanceOf[Int]) ??? 
    else ???

or the particular value: 

In [ ]:
def foo[A](a: A): A = 
    if (a.toString == "") ??? 
    else if (a.hashCode == 0) ??? 
    else if (a.equals(1.0)) ???
    else ???

But, actually, in logical justice, just from the meaning of the signature, we can only do one thing: 

In [ ]:
def foo[A](a: A): A = a

![knownothing](../misc/knownothing.jpg)

This is a most relevant feature of parametric polymorphism: we don't know anything about generic parameters. This sole fact retricts our implementations a lot, and, accordingly, makes the corresponding signatures very expressive. 

Another example. There are many different implementations of this function:  

In [ ]:
def apply(f: Int => String, i: Int): String = 
    ???

But only one for its generic version:

In [ ]:
def apply[A, B](f: A => B, a: A): B = 
    f(a)

Yet another example:

In [ ]:
def andThen(f1: Int => String, f2: String => Boolean): Int => Boolean = 
    (i: Int) => ??? : Boolean

What about its generic version?

In [ ]:
def andThen[A, B, C](f1: A => B, f2: B => C): A => C = ???

This is only a little bit more complicated, but let's ask lambda man for help!

![lambdamanhelp](../misc/lambdamanhelp.png)

In [ ]:
def andThen[A, B, C](f1: A => B, f2: B => C): A => C = 
    (a: A) => f2(f1(a : A) : B) : C

Of course, It's not always the case that there is only one possible implementation. In the following case, we may rearrange the list values in multiple ways: 

In [ ]:
def foo[A](list: List[A]): List[A] = 
    ???

But we can be sure that any implementation of `foo` must satisfy the following property: 

![freetheorems](../misc/lambdamanfreetheorems.png)

In [ ]:
def map[A, B](f: A => B): List[A] => List[B] = ???

In [ ]:
def property[A, B](f: A => B, x: List[A]): Boolean = 
    ((foo[A](_)) andThen map[A, B](f))(x) == 
    (map[A, B](f) andThen (foo[B](_)))(x)

# Generics and subtyping

Let's turn now to explain the interactions between generics and subtyping. We will use the following inheritance hierarchy:

In [1]:
class Complex(val real: Double, val img: Double)
class Real(val value: Double)              extends Complex(value, 0.0)
class Rational(val num: Int, val den: Int) extends Real(num/den)
class IntegerN(val int: Int)               extends Rational(int,1)
class Natural(val nat: Int)                extends IntegerN(nat.abs)

defined class Complex
defined class Real
defined class Rational
defined class IntegerN
defined class Natural

In [2]:
val `1+2i`: Complex = new Complex(1.0,2.0)
val π: Real = new Real(3.1415)
val `2/3`: Rational = new Rational(2, 3)
val `-1`: IntegerN = new IntegerN(-1)
val `1`: Natural = new Natural(1)

`1+2i`: Complex = ammonite.$sess.cmd0$Helper$Complex@7596bf98
π: Real = ammonite.$sess.cmd0$Helper$Real@66d2a11f
`2/3`: Rational = ammonite.$sess.cmd0$Helper$Rational@3a318c06
`-1`: IntegerN = ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989
`1`: Natural = ammonite.$sess.cmd0$Helper$Natural@593ff3ac

According to this hierarchy, naturals are integers, integers are rational numbers, and so on. This means that we can use a natural number whenever we need an integer, e.g. in passing a value to a function, initialising a variable, etc. Thus, given function `foo` expecting a `Rational` number: 

In [3]:
def foo(c: Rational): Rational = c

defined function foo

we may safely pass a rational, integer or natural number: 

In [4]:
foo(`2/3`)
foo(`-1`)
foo(`1`)

res3_0: Rational = ammonite.$sess.cmd0$Helper$Rational@3a318c06
res3_1: Rational = ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989
res3_2: Rational = ammonite.$sess.cmd0$Helper$Natural@593ff3ac

but not a real or complex number: 

In [4]:
// foo(`1+2i`)
// foo(π)

### Covariant and contravariant parameters

Let's now investigate which subtyping relationships hold between functions. For instance, given the following higher-order function:

In [5]:
def foo(f: Rational => Rational): Rational = 
    f(new Rational(4,2))

defined function foo

we can, of course, pass this function:

In [6]:
val times2: Rational => Rational = 
    r => new Rational(r.num*2,r.den)

times2: Rational => Rational = ammonite.$sess.cmd5$Helper$$Lambda$2119/571520661@73f5efdd

In [7]:
foo(times2)

res6: Rational = ammonite.$sess.cmd0$Helper$Rational@15b16fc3

But, can we pass this function?

In [8]:
val idiv: Rational => IntegerN = 
    r => new IntegerN(r.num/r.den)

idiv: Rational => IntegerN = ammonite.$sess.cmd7$Helper$$Lambda$2126/2004944453@482f58c5

In [9]:
foo(idiv)

res8: Rational = ammonite.$sess.cmd0$Helper$IntegerN@679978d6

And what about this one?

In [10]:
val rr2: Real => Rational = 
    _ => new Rational(0,1)

rr2: Real => Rational = ammonite.$sess.cmd9$Helper$$Lambda$2132/296904236@7df0cd8d

In [11]:
foo(rr2)

res10: Rational = ammonite.$sess.cmd0$Helper$Rational@1f1c8ff8

This makes sense. In the former case, we pass a function which returns a value of a more specific type, `Integer`, which is perfectly compatible with the demanded type, `Rational`. In the latter one, we pass a function that is able to be applied to `Real` values, and, in particular, to `Rational`s, which is what will happen. So, no problem at all.

What about this function?

In [12]:
val rr3: Natural => Rational = _ => new Rational(0,1)

rr3: Natural => Rational = ammonite.$sess.cmd11$Helper$$Lambda$2138/12433853@276d7464

In [12]:
//foo(rr3)

And for the final case:

In [13]:
val rr4: Rational => Complex = _ => new Complex(0.0, 0.0)

rr4: Rational => Complex = ammonite.$sess.cmd12$Helper$$Lambda$2142/1985443353@1ecaff08

In [13]:
// foo(rr4)

Neither case work, and it make sense. In the former one, we pass a function that can only receive naturals, but in the body of `apply` we see that it will be applied to a rational number. In the second case, `apply` is given a function that returns a complex value, whereas it can only handle rationals.

Let's see now how Scala defines `Function1` so that the previous subtyping relationships hold. We may abstract the following monomorphic function from the input and output types:

In [14]:
trait Function1IntToString{
    def apply(i: Int): String
}

defined trait Function1IntToString

and obtain the following definition:

In [15]:
trait Function1[I, O]{
    def apply(i: I): O
}

defined trait Function1

This is a good start. Let's redefine our `foo` function and attempt to reproduce the desired behaviour.

In [16]:
def foo(f: Function1[Rational, Rational]): Rational = 
    f(`2/3`)

defined function foo

Of course, it work for the obvious case:

In [17]:
val times2: Function1[Rational, Rational] = 
    new Function1[Rational, Rational]{
        def apply(r: Rational): Rational = 
            new Rational(r.num*2,r.den)
    }

times2: Function1[Rational, Rational] = ammonite.$sess.cmd16$Helper$$anon$1@6ddcd73d

In [18]:
foo(times2)

res17: Rational = ammonite.$sess.cmd0$Helper$Rational@6b811390

But it fails when the return type is a subtype:

In [19]:
val idiv: Function1[Rational, IntegerN] = 
    new Function1[Rational, IntegerN]{
        def apply(r: Rational): IntegerN = 
            new IntegerN(r.num/r.den)
    }

idiv: Function1[Rational, IntegerN] = ammonite.$sess.cmd18$Helper$$anon$1@21662c9

In [19]:
// foo(idiv)

The compiler error gives us a definitive clue: we have to declare the second parameter type of `Function1` *covariantly* :

In [21]:
trait Function1[I, +O]{
    def apply(i: I): O
}

def foo(f: Function1[Rational, Rational]): Rational = 
    f(`2/3`)

val idiv: Function1[Rational, IntegerN] = 
    new Function1[Rational, IntegerN]{
        def apply(r: Rational): IntegerN = 
            new IntegerN(r.num/r.den)
    }

defined trait Function1
defined function foo
idiv: Function1[Rational, IntegerN] = ammonite.$sess.cmd20$Helper$$anon$1@7a7f0055

In [22]:
foo(idiv)

res21: Rational = ammonite.$sess.cmd0$Helper$IntegerN@1cfde3d6

We still have a problem, however: 

In [23]:
val rr2 = new Function1[Real, Rational]{
    def apply(r: Real): Rational = 
        new Rational(0,1)
}

rr2: AnyRef with Function1[Real, Rational] = ammonite.$sess.cmd22$Helper$$anon$1@42e4b58b

In [23]:
// foo(rr2)

In this case we need to make the first type parameter *contravariant* : 

In [24]:
trait Function1[-I, +O]{
    def apply(i: I): O
}

def foo(f: Function1[Rational, Rational]): Rational = 
    f(`2/3`)

val idiv: Function1[Rational, IntegerN] = 
    new Function1[Rational, IntegerN]{
        def apply(r: Rational): IntegerN = 
            new IntegerN(r.num/r.den)
    }

val rr2 = new Function1[Real, Rational]{
    def apply(r: Real): Rational = 
        new Rational(0,1)
}

defined trait Function1
defined function foo
idiv: Function1[Rational, IntegerN] = ammonite.$sess.cmd23$Helper$$anon$1@7ffd329
rr2: AnyRef with Function1[Real, Rational] = ammonite.$sess.cmd23$Helper$$anon$2@29173ccf

And all the cases work as expected:

In [25]:
foo(idiv)
foo(rr2)

res24_0: Rational = ammonite.$sess.cmd0$Helper$IntegerN@7253b475
res24_1: Rational = ammonite.$sess.cmd0$Helper$Rational@438a4eac

### Subtype bounds

Given the following generic definition of lists:

In [27]:
sealed abstract class MyList[A]
case class End[A]() extends MyList[A]
case class Cons[A](head: A, tail: MyList[A]) extends MyList[A]

defined class MyList
defined class End
defined class Cons

We can create a list of rationals with rational numbers, but not with reals or complex numbers: 

In [28]:
val a: MyList[Rational] = 
    Cons(`2/3`, Cons(`-1`, Cons(`1`, End())))

a: MyList[Rational] = Cons(
  ammonite.$sess.cmd0$Helper$Rational@3a318c06,
  Cons(
    ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989,
    Cons(ammonite.$sess.cmd0$Helper$Natural@593ff3ac, End())
  )
)

Similar question than before: which kinds of lists would be safe to pass to this function?

In [29]:
def `prepend_2/3`(l: MyList[Rational]): MyList[Rational] = 
    Cons(`2/3`, l)

defined function `prepend_2/3`

We may of course pass a list of rationals: 

In [31]:
val ratList: MyList[Rational] = Cons(`1`, Cons(`-1`, End()))
`prepend_2/3`(ratList)

ratList: MyList[Rational] = Cons(
  ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
  Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
)
res30_1: MyList[Rational] = Cons(
  ammonite.$sess.cmd0$Helper$Rational@3a318c06,
  Cons(
    ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
    Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
  )
)

But we can't pass a list of integers, which it's a pity: 

In [32]:
val intList: MyList[IntegerN] = Cons(`1`, Cons(`-1`, End()))
// `prepend_2/3`(intList)

intList: MyList[IntegerN] = Cons(
  ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
  Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
)

The very same compiler gives us a clue to solve the problem: "You may wish to define A as +A instead." Let's listen to it and make that change: 

In [49]:
sealed abstract class MyList[+A]
case class End[A]() extends MyList[A]
case class Cons[A](head: A, tail: MyList[A]) extends MyList[A]

def `prepend_2/3`(l: MyList[Rational]): MyList[Rational] = 
    Cons(`2/3`, l)

val intList: MyList[IntegerN] = Cons(`1`, Cons(`-1`, End()))

defined class MyList
defined class End
defined class Cons
defined function `prepend_2/3`
intList: MyList[IntegerN] = Cons(
  ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
  Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
)

and now it works:

In [34]:
`prepend_2/3`(intList)

intList: MyList[IntegerN] = Cons(
  ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
  Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
)
res33_1: MyList[Rational] = Cons(
  ammonite.$sess.cmd0$Helper$Rational@3a318c06,
  Cons(
    ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
    Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
  )
)

How can we generalise the function `prepend_2/3`?  This should be ok:

In [44]:
def prepend[N](l: MyList[N], r: N): MyList[N] = 
    Cons(r, l)

defined function prepend

Indeed, it supports those cases in which the actual types of `r` and `l` are different but compatible. For instance:

In [45]:
prepend(intList, π) // since MyList[Int] <: MyList[Real]
prepend(intList, `1`: Natural) // since Natural <: Int

res44_0: MyList[Real] = Cons(
  ammonite.$sess.cmd0$Helper$Real@66d2a11f,
  Cons(
    ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
    Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
  )
)
res44_1: MyList[IntegerN] = Cons(
  ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
  Cons(
    ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
    Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
  )
)

If we intend, however, to implement this function as a method class, we run into trouble:

In [38]:
sealed abstract class MyList[+A]{
    //def prepend(a: A): MyList[A] = 
      //  Cons(a, this)
}
case class End[A]() extends MyList[A]
case class Cons[A](head: A, tail: MyList[A]) extends MyList[A]

defined class MyList
defined class End
defined class Cons

The problem is that, now, the type of the list is fixed in the method invocation. This implies that if we want to allow for the resulting list type to be more generic, we have to twist the method declaration a little bit with a so-called *supertype bound*:

In [50]:
sealed abstract class MyList[+A]{
    def prepend[B >: A](a: B): MyList[B] = 
        Cons(a, this)
}
case class End[A]() extends MyList[A]
case class Cons[A](head: A, tail: MyList[A]) extends MyList[A]

val intList: MyList[IntegerN] = Cons(`1`, Cons(`-1`, End()))

defined class MyList
defined class End
defined class Cons
intList: MyList[IntegerN] = Cons(
  ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
  Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
)

And now everything works:

In [52]:
intList.prepend[IntegerN](`1`)
intList.prepend[Real](π)

res51_0: MyList[IntegerN] = Cons(
  ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
  Cons(
    ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
    Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
  )
)
res51_1: MyList[Real] = Cons(
  ammonite.$sess.cmd0$Helper$Real@66d2a11f,
  Cons(
    ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
    Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
  )
)

Last, similarly to super-type bounds, we may also use subtype bounds, as in the following declaration, where we only allow the method to work for `Rational`s:

In [56]:
def prependS[N <: Rational](l: MyList[N], r: N): MyList[N] = 
    Cons(r, l)

defined function prependS

Now, if we pass a list of integers it will work in the same way as in the previous definition of `prepend`: 

In [57]:
prependS(intList,`-1`)

res56: MyList[IntegerN] = Cons(
  ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989,
  Cons(
    ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
    Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
  )
)

but it will fail with lists of reals:

In [57]:
// prependS(Cons(π, End()), π)

But, which is the difference of `prependS` with the following monomorphic function?

In [58]:
def prependR(l: MyList[Rational], r: Rational): MyList[Rational] = 
    Cons(r, l)

defined function prependR

Certainly, this function will allow us to pass the same types of values:

In [59]:
prependR(intList: MyList[IntegerN], `1`: Natural)

res58: MyList[Rational] = Cons(
  ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
  Cons(
    ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
    Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
  )
)

but note that the resulting type is too general, compared to the one obtained here:

In [60]:
prependS(intList: MyList[IntegerN], `1`: Natural)

res59: MyList[IntegerN] = Cons(
  ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
  Cons(
    ammonite.$sess.cmd0$Helper$Natural@593ff3ac,
    Cons(ammonite.$sess.cmd0$Helper$IntegerN@2a5fe989, End())
  )
)

# Futher topics

There are many topics related to generics we may talk about (for some hours more :):
* Higher-kinded generics. So-called type constructor parameters. 
* The curry-howard isomorphism. Showing how generic signatures correspond to propositions of second-order logic, and their implementations to proofs.
* Parametric definitions of ADTs. How we can give isomorphic definitions of tuples and eithers without data types at all.
* Generics in dotty. Improvements over the Scala compiler related to generics. 

# Conclusion

Some takeaways: 
* Generic parameters are additional parameters of types and functions, even if you don't see them in method invocations or instance declarations!
* Use them whenever you want your code to be more modular and more reusable. 
* Take generic signatures seriously, and don't cheat (even if the Scala compiler allows you to be impure)!
* Variance and contravariance annotations are difficult to reason about. Try to avoid them ... by not using inheritance!